# The Complete Pipeline: ROS Deployment + Gazebo Visualization

**← [Back to What is PyKal?](../what_is_pykal/index.rst)** | **[Previous: ROS Deployment](ros_deployment.ipynb)**

---

This is the **culminating notebook** of the PyKal tutorial series. We bring together everything:

1. ✅ **Theory**: Kalman filter + PID control (DynamicalSystems)
2. ✅ **Software**: Pure Python implementation
3. ✅ **Simulation**: Corruption + preparation pipeline
4. ✅ **ROS Deployment**: Distributed node architecture
5. ✅ **Hardware Simulation**: Gazebo with real physics
6. ✅ **Interactive Control**: Keyboard teleoperation
7. ✅ **Live Visualization**: Real-time performance monitoring

## What We'll Build

A **complete robotic system** where you:
- Control TurtleBot3's velocity with **keyboard** (↑ increases setpoint, ↓ decreases)
- See the robot move in **Gazebo** with realistic physics
- **Corrupt** sensor data (noise, bias, drift, dropouts)
- **Filter** with Kalman filter using data preparation
- **Control** with PID to track your setpoint
- **Monitor** everything with real-time plots and diagnostics

This is what hardware deployment looks like - but you can iterate in simulation first!

**Press `Ctrl+C` in the terminal to stop when done.**

## Imports and Setup

In [ ]:
from pykal import DynamicalSystem, ROSNode
from pykal.data_change import corrupt, prepare
from pykal.gazebo import start_gazebo, stop_gazebo
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time
import threading
from collections import defaultdict, deque
from IPython.display import clear_output

# For ROS message types
from std_msgs.msg import Float64
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

# For keyboard input
try:
    from pynput import keyboard
    KEYBOARD_AVAILABLE = True
except ImportError:
    print("⚠ pynput not available. Install with: pip install pynput")
    print("   Falling back to programmatic setpoint changes.")
    KEYBOARD_AVAILABLE = False

# Set random seed
np.random.seed(42)

# Configure matplotlib
try:
    # Try widget backend for interactive Jupyter
    get_ipython().run_line_magic('matplotlib', 'widget')
except:
    # Fall back to inline for automated testing
    matplotlib.use('Agg')
    
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 9

print("✓ Imports complete. Ready for deployment!")

## Launch Gazebo Simulation

Start TurtleBot3 in Gazebo. This gives us a real physics simulation to test our control system.

In [ ]:
# Launch Gazebo with TurtleBot3
gz = start_gazebo(
    robot='turtlebot3',
    world='turtlebot3_world',
    headless=False,  # Show GUI so we can see the robot
    x_pose=0.0,
    y_pose=0.0,
    yaw=0.0,
    model='burger'
)

print(f"\n{'='*60}")
print(f"Gazebo Status: {gz}")
print(f"{'='*60}")
print("\n✓ TurtleBot3 spawned and ready!")
print("  Check the Gazebo window to see your robot.")
print(f"\n{'='*60}\n")

## System Parameters

Same control parameters as the simulation, adapted for TurtleBot3's velocity control.

In [ ]:
# PID gains (tuned for velocity control)
KP = 1.5
KI = 0.2
KD = 0.5

# Kalman filter parameters
Q = 0.01  # Process noise
R = 0.05  # Measurement noise

# Control loop rate
rate_hz = 20.0  # 20 Hz control loop

# Initial setpoint
initial_setpoint = 0.0  # Start stationary
setpoint_increment = 0.05  # m/s per keypress

print(f"✓ Parameters configured:")
print(f"  PID: KP={KP}, KI={KI}, KD={KD}")
print(f"  Kalman: Q={Q}, R={R}")
print(f"  Control rate: {rate_hz} Hz")
print(f"  Setpoint increment: {setpoint_increment} m/s per keypress")

## Keyboard Control Setup

Use arrow keys to control the velocity setpoint:
- **↑ (Up Arrow)**: Increase setpoint (go faster)
- **↓ (Down Arrow)**: Decrease setpoint (go slower / reverse)
- **Space**: Reset to zero
- **ESC**: Stop control (press Ctrl+C in terminal to fully exit)

In [ ]:
# Shared state for keyboard input
keyboard_state = {
    'setpoint_delta': 0.0,
    'reset': False,
    'active': True,
}
keyboard_lock = threading.Lock()

if KEYBOARD_AVAILABLE:
    def on_press(key):
        """Handle key press events."""
        with keyboard_lock:
            try:
                if key == keyboard.Key.up:
                    keyboard_state['setpoint_delta'] = setpoint_increment
                    print(f"↑ Increasing setpoint (+{setpoint_increment:.2f} m/s)")
                elif key == keyboard.Key.down:
                    keyboard_state['setpoint_delta'] = -setpoint_increment
                    print(f"↓ Decreasing setpoint (-{setpoint_increment:.2f} m/s)")
                elif key == keyboard.Key.space:
                    keyboard_state['reset'] = True
                    print("⏸ Reset to zero")
                elif key == keyboard.Key.esc:
                    keyboard_state['active'] = False
                    print("⏹ ESC pressed - stopping control loop")
                    return False  # Stop listener
            except AttributeError:
                pass

    def on_release(key):
        """Handle key release events."""
        with keyboard_lock:
            # Reset delta when key is released
            if key in [keyboard.Key.up, keyboard.Key.down]:
                keyboard_state['setpoint_delta'] = 0.0

    # Start keyboard listener in background thread
    listener = keyboard.Listener(on_press=on_press, on_release=on_release)
    listener.start()
    
    print("✓ Keyboard listener started!")
    print("  Use arrow keys to control:")
    print("    ↑ = Increase velocity setpoint")
    print("    ↓ = Decrease velocity setpoint")
    print("    SPACE = Reset to zero")
    print("    ESC = Stop control loop")
else:
    print("✓ Programmatic setpoint control enabled")
    print("  (Install pynput for keyboard control)")

## Control System Components

Same DynamicalSystem definitions, adapted for velocity control instead of the abstract car model.

In [ ]:
# === PID Controller ===
def controller_f(ck, rk, xhat_k):
    """Update controller state (error history and integral)."""
    ek_prev, Ik, _ = ck
    ek = rk - xhat_k
    Ik_next = Ik + ek
    return (ek, Ik_next, ek_prev)

def controller_h(ck, rk, xhat_k, KP, KI, KD):
    """Compute PID control output."""
    ek_prev, Ik_next, ek_old = ck
    ek = rk - xhat_k
    uk = KP * ek + KI * Ik_next + KD * (ek - ek_old)
    return uk

controller_sys = DynamicalSystem(f=controller_f, h=controller_h, state_name="ck")

# === Observer (Kalman Filter for velocity) ===
def observer_f(xhat_P, yk, uk_prev, dt, Q, R):
    """Simple velocity Kalman filter (nearly constant velocity model)."""
    xhat_prev, P_prev = xhat_P
    
    # Simplified dynamics: velocity changes based on control input
    # This is a rough model - in reality TurtleBot dynamics are more complex
    F = 0.9  # Velocity decay factor (friction)
    B = 0.1  # Control effectiveness
    H = 1.0  # Direct measurement
    
    # Predict
    xhat_pred = F * xhat_prev + B * uk_prev
    P_pred = F * P_prev * F + Q
    
    # Update
    S = H * P_pred * H + R
    K = P_pred * H / (S + 1e-6)
    innovation = yk - H * xhat_pred
    xhat_upd = xhat_pred + K * innovation
    P_upd = (1 - K * H) * P_pred
    
    return (xhat_upd, P_upd)

def observer_h(xhat_P):
    """Extract velocity estimate."""
    xhat, _ = xhat_P
    return xhat

observer_sys = DynamicalSystem(f=observer_f, h=observer_h, state_name="xhat_P")

print("✓ DynamicalSystems created (PID + Kalman filter)")

## Data Logger

Record all data streams for visualization. We'll use a circular buffer for efficient real-time plotting.

In [ ]:
# Global storage with circular buffers for real-time plotting
BUFFER_SIZE = 500  # Keep last 500 samples
logged_data = {
    'time': deque(maxlen=BUFFER_SIZE),
    'setpoint': deque(maxlen=BUFFER_SIZE),
    'true_velocity': deque(maxlen=BUFFER_SIZE),
    'measurement': deque(maxlen=BUFFER_SIZE),
    'estimate': deque(maxlen=BUFFER_SIZE),
    'control': deque(maxlen=BUFFER_SIZE),
}
data_lock = threading.Lock()

def logger_callback(tk, setpoint=None, true_velocity=None, measurement=None,
                   estimate=None, control=None):
    """Log all data with timestamps."""
    with data_lock:
        logged_data['time'].append(tk)
        logged_data['setpoint'].append(setpoint[0] if setpoint is not None else np.nan)
        logged_data['true_velocity'].append(true_velocity[0] if true_velocity is not None else np.nan)
        logged_data['measurement'].append(measurement[0] if measurement is not None else np.nan)
        logged_data['estimate'].append(estimate[0] if estimate is not None else np.nan)
        logged_data['control'].append(control[0] if control is not None else np.nan)
    return {}

logger_node = ROSNode(
    node_name='data_logger',
    callback=logger_callback,
    subscribes_to=[
        ('/setpoint', Float64, 'setpoint'),
        ('/true_velocity', Float64, 'true_velocity'),
        ('/measurement', Float64, 'measurement'),
        ('/estimate', Float64, 'estimate'),
        ('/control', Float64, 'control'),
    ],
    rate_hz=rate_hz,
)

print("✓ Logger node created")

## ROS Node Network

Create the distributed control system:
1. **Setpoint Generator**: Keyboard-controlled setpoint
2. **Sensor Corruption**: Corrupt odometry from Gazebo
3. **Kalman Filter**: Estimate velocity with data preparation
4. **PID Controller**: Generate control commands
5. **Command Publisher**: Send Twist commands to TurtleBot

In [ ]:
# === Setpoint Generator (Keyboard Controlled) ===
setpoint_state = {'current': initial_setpoint}

def setpoint_callback(tk):
    """Generate setpoint based on keyboard input."""
    with keyboard_lock:
        delta = keyboard_state['setpoint_delta']
        reset = keyboard_state['reset']
        keyboard_state['reset'] = False  # Clear reset flag
    
    if reset:
        setpoint_state['current'] = 0.0
    else:
        setpoint_state['current'] += delta * (1.0 / rate_hz)  # Integrate delta
    
    # For programmatic control (if keyboard not available)
    if not KEYBOARD_AVAILABLE:
        # Simple pattern for demo
        if tk < 5.0:
            setpoint_state['current'] = 0.2
        elif tk < 10.0:
            setpoint_state['current'] = 0.4
        elif tk < 15.0:
            setpoint_state['current'] = 0.2
        else:
            setpoint_state['current'] = 0.0
    
    return {'setpoint': np.array([setpoint_state['current']])}

setpoint_node = ROSNode(
    node_name='setpoint_generator',
    callback=setpoint_callback,
    publishes_to=[('setpoint', Float64, '/setpoint')],
    rate_hz=rate_hz,
)

print("✓ Setpoint generator created (keyboard controlled)")

In [ ]:
# === Sensor Corruption Node ===
corruption_state = {'step_count': 0}

def corruption_callback(tk, odom=None):
    """Corrupt odometry data from Gazebo."""
    if odom is None:
        return {'true_velocity': np.array([0.0]), 'measurement': np.array([np.nan])}
    
    # Extract linear velocity from odometry (odom is [x, y, z, vx, vy, vz, ...])
    true_vel = odom[3]  # vx component
    
    # === APPLY CORRUPTION ===
    i = corruption_state['step_count']
    corrupted_vel = true_vel
    
    # 1. Sensor noise (realistic for IMU/wheel encoders)
    corrupted_vel = corrupt.with_gaussian_noise(
        np.array([corrupted_vel]), std=0.03, seed=42+i
    )[0]
    
    # 2. Sensor bias (calibration error)
    corrupted_vel = corrupt.with_bias(
        np.array([corrupted_vel]), bias=0.02
    )[0]
    
    # 3. Occasional dropouts (5% - sensor glitches)
    if np.random.rand() < 0.05:
        corrupted_vel = np.nan
    
    corruption_state['step_count'] += 1
    
    return {
        'true_velocity': np.array([true_vel]),
        'measurement': np.array([corrupted_vel]),
    }

corruption_node = ROSNode(
    node_name='sensor_corruption',
    callback=corruption_callback,
    subscribes_to=[('/odom', Odometry, 'odom')],
    publishes_to=[
        ('true_velocity', Float64, '/true_velocity'),
        ('measurement', Float64, '/measurement'),
    ],
    rate_hz=rate_hz,
    stale_config={'odom': {'after': 1.0, 'policy': 'hold'}},
)

print("✓ Sensor corruption node created")

In [ ]:
# === Kalman Filter Node (with preparation) ===
observer_state = {'xhat_P': (0.0, 1.0), 'uk_prev': 0.0, 'yk_prev': 0.0}

def observer_callback(tk, measurement=None, control=None):
    """Kalman filter with data preparation."""
    yk_raw = measurement[0] if measurement is not None else np.nan
    uk = control[0] if control is not None else 0.0
    
    # === APPLY PREPARATION ===
    yk_prepared = yk_raw
    
    # 1. Handle dropouts with hold policy
    if np.isnan(yk_prepared):
        yk_prepared = observer_state['yk_prev']
    
    # 2. Remove bias (calibration)
    yk_prepared = yk_prepared - 0.02
    
    yk = yk_prepared
    observer_state['yk_prev'] = yk
    
    # Update Kalman filter
    xhat_P = observer_state['xhat_P']
    uk_prev = observer_state['uk_prev']
    
    xhat_P_new = observer_sys.f(
        xhat_P=xhat_P, yk=yk, uk_prev=uk_prev,
        dt=1.0/rate_hz, Q=Q, R=R
    )
    xhat = observer_sys.h(xhat_P_new)
    
    observer_state['xhat_P'] = xhat_P_new
    observer_state['uk_prev'] = uk
    
    return {'estimate': np.array([xhat])}

observer_node = ROSNode(
    node_name='kalman_filter',
    callback=observer_callback,
    subscribes_to=[
        ('/measurement', Float64, 'measurement'),
        ('/control', Float64, 'control'),
    ],
    publishes_to=[('estimate', Float64, '/estimate')],
    rate_hz=rate_hz,
    stale_config={
        'measurement': {'after': 0.5, 'policy': 'hold'},
        'control': {'after': 0.5, 'policy': 'hold'},
    },
    enable_diagnostics=True,
)

print("✓ Kalman filter node created (with preparation)")

In [ ]:
# === PID Controller Node ===
controller_state = {'ck': (0.0, 0.0, 0.0)}

def controller_callback(tk, setpoint=None, estimate=None):
    """PID controller for velocity tracking."""
    if setpoint is None or estimate is None:
        return {'control': np.array([0.0])}
    
    rk = setpoint[0]
    xhat_k = estimate[0]
    
    # Update controller
    ck = controller_state['ck']
    ck_new, uk = controller_sys.step(
        return_state=True,
        param_dict={'ck': ck, 'rk': rk, 'xhat_k': xhat_k, 'KP': KP, 'KI': KI, 'KD': KD}
    )
    controller_state['ck'] = ck_new
    
    return {'control': np.array([uk])}

controller_node = ROSNode(
    node_name='pid_controller',
    callback=controller_callback,
    subscribes_to=[
        ('/setpoint', Float64, 'setpoint'),
        ('/estimate', Float64, 'estimate'),
    ],
    publishes_to=[('control', Float64, '/control')],
    rate_hz=rate_hz,
    required_topics={'setpoint', 'estimate'},
    stale_config={
        'setpoint': {'after': 0.5, 'policy': 'hold'},
        'estimate': {'after': 0.5, 'policy': 'hold'},
    },
)

print("✓ PID controller node created")

In [ ]:
# === Command Publisher Node ===
# Maps control signal to Twist commands for TurtleBot

def command_callback(tk, control=None):
    """Convert control signal to Twist command."""
    if control is None:
        uk = 0.0
    else:
        uk = control[0]
    
    # Clip control signal to safe range
    uk = np.clip(uk, -0.5, 0.5)  # Max 0.5 m/s
    
    # Create Twist message [linear.x, linear.y, linear.z, angular.x, angular.y, angular.z]
    twist = np.array([uk, 0.0, 0.0, 0.0, 0.0, 0.0])
    
    return {'cmd_vel': twist}

command_node = ROSNode(
    node_name='command_publisher',
    callback=command_callback,
    subscribes_to=[('/control', Float64, 'control')],
    publishes_to=[('cmd_vel', Twist, '/cmd_vel')],
    rate_hz=rate_hz,
    stale_config={'control': {'after': 0.2, 'policy': 'zero'}},  # Safety: zero if stale
)

print("✓ Command publisher node created")

## Start the System

Launch all nodes and start the control loop. Watch Gazebo to see your robot respond to commands!

**Use the arrow keys to control the robot's velocity setpoint.**

In [ ]:
print("\n" + "="*60)
print("STARTING ROS NODE NETWORK")
print("="*60)

# Start all nodes
nodes = [
    ('Setpoint Generator', setpoint_node),
    ('Sensor Corruption', corruption_node),
    ('Kalman Filter', observer_node),
    ('PID Controller', controller_node),
    ('Command Publisher', command_node),
    ('Data Logger', logger_node),
]

for name, node in nodes:
    node.create_node()
    node.start()
    print(f"  ✓ {name}")

print("="*60)
print(f"\n✓ All nodes running!")
print(f"\nControl Instructions:")
if KEYBOARD_AVAILABLE:
    print(f"  ↑ = Increase velocity")
    print(f"  ↓ = Decrease velocity")
    print(f"  SPACE = Stop (zero velocity)")
    print(f"  ESC = Exit control loop")
else:
    print(f"  (Programmatic control - install pynput for keyboard)")
print(f"\nWatch the Gazebo window to see your robot move!")
print(f"Run the next cell for real-time monitoring plots.")
print(f"\nPress Ctrl+C in terminal to stop everything.")
print("="*60 + "\n")

# For automated testing: collect data for a short period
if not KEYBOARD_AVAILABLE:
    print("Collecting data for 20 seconds (automated mode)...")
    time.sleep(20)
    print("✓ Data collection complete")

## Real-Time Performance Monitoring

Live plots showing:
1. Setpoint tracking (how well the controller follows your commands)
2. Sensor corruption vs filtered estimate
3. Control signal output

**Let this run for ~30 seconds to see the system in action.**

In [ ]:
# Real-time visualization
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
plt.tight_layout(pad=3.0)

# Initialize plots
line_setpoint, = axes[0].plot([], [], 'k--', linewidth=2, label='Setpoint')
line_true, = axes[0].plot([], [], 'g-', linewidth=2, alpha=0.7, label='True Velocity')
line_estimate, = axes[0].plot([], [], 'b-', linewidth=1.5, label='Estimate (KF)')
axes[0].set_ylabel('Velocity (m/s)')
axes[0].set_title('Real-Time Velocity Tracking', fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.6, 0.6])

scatter_meas = axes[1].scatter([], [], c='red', s=10, alpha=0.4, label='Corrupted Meas.')
line_true2, = axes[1].plot([], [], 'g-', linewidth=2, alpha=0.5, label='True Velocity')
line_estimate2, = axes[1].plot([], [], 'b-', linewidth=1.5, label='Filtered Est.')
axes[1].set_ylabel('Velocity (m/s)')
axes[1].set_title('Corruption + Filtering', fontweight='bold', color='red')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.6, 0.6])

line_control, = axes[2].plot([], [], 'purple', linewidth=1.5, label='Control Signal')
axes[2].axhline(y=0, color='k', linestyle=':', alpha=0.3)
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('Control')
axes[2].set_title('PID Controller Output', fontweight='bold')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([-1.0, 1.0])

def update_plot(frame):
    """Update plots with latest data."""
    with data_lock:
        if len(logged_data['time']) == 0:
            return
        
        t = np.array(logged_data['time'])
        setpoint = np.array(logged_data['setpoint'])
        true_vel = np.array(logged_data['true_velocity'])
        measurement = np.array(logged_data['measurement'])
        estimate = np.array(logged_data['estimate'])
        control = np.array(logged_data['control'])
    
    # Update time window
    t_min = max(0, t[-1] - 20)  # Show last 20 seconds
    t_max = t[-1] + 1
    
    for ax in axes:
        ax.set_xlim([t_min, t_max])
    
    # Plot 1: Tracking
    line_setpoint.set_data(t, setpoint)
    line_true.set_data(t, true_vel)
    line_estimate.set_data(t, estimate)
    
    # Plot 2: Corruption
    valid_mask = ~np.isnan(measurement)
    scatter_meas.set_offsets(np.c_[t[valid_mask], measurement[valid_mask]])
    line_true2.set_data(t, true_vel)
    line_estimate2.set_data(t, estimate)
    
    # Plot 3: Control
    line_control.set_data(t, control)
    
    return [line_setpoint, line_true, line_estimate, scatter_meas,
            line_true2, line_estimate2, line_control]

# Start animation
print("Starting real-time visualization...")
print("(Let this run for ~30 seconds to collect data)\n")

anim = FuncAnimation(fig, update_plot, interval=200, blit=True, cache_frame_data=False)
plt.show()

print("\n✓ Real-time monitoring active")
print("  Watch the plots update as you control the robot!")

## System Diagnostics

Check the Kalman filter node's diagnostics to see message rates, latency, and errors.

In [ ]:
diag = observer_node.get_diagnostics()

print("\n" + "="*60)
print("KALMAN FILTER NODE DIAGNOSTICS")
print("="*60)
print(f"\nUptime: {diag['uptime_seconds']:.1f}s")
print(f"Callbacks executed: {diag['callback_count']}")
print(f"Callback rate: {diag['callback_rate_hz']:.1f} Hz")
print(f"Callback errors: {diag['callback_errors']}")

print("\nTopic Statistics:")
for topic, stats in diag['topics'].items():
    print(f"\n  {topic}:")
    print(f"    Messages received: {stats['msg_count']}")
    print(f"    Message rate: {stats['rate_hz']:.1f} Hz")
    print(f"    Conversion errors: {stats['error_count']}")
    print(f"    Last message age: {stats['last_msg_age']:.3f}s")
    latency = stats.get('latency_ms')
    if latency is not None:
        print(f"    Average latency: {latency:.1f} ms")

print("\n" + "="*60 + "\n")

## Final Summary Plots

Comprehensive visualization of the complete test run.

In [ ]:
# Stop the animation first
try:
    anim.event_source.stop()
except:
    pass

# Create final summary plots
with data_lock:
    t = np.array(logged_data['time'])
    setpoint = np.array(logged_data['setpoint'])
    true_vel = np.array(logged_data['true_velocity'])
    measurement = np.array(logged_data['measurement'])
    estimate = np.array(logged_data['estimate'])
    control = np.array(logged_data['control'])

fig_summary, axes_summary = plt.subplots(4, 1, figsize=(14, 14))
plt.tight_layout(pad=3.0)

# Plot 1: Complete tracking history
axes_summary[0].plot(t, setpoint, 'k--', linewidth=2, label='Setpoint (Keyboard)')
axes_summary[0].plot(t, true_vel, 'g-', linewidth=2, alpha=0.7, label='True Velocity')
axes_summary[0].plot(t, estimate, 'b-', linewidth=1.5, label='Estimate (KF)')
axes_summary[0].set_ylabel('Velocity (m/s)')
axes_summary[0].set_title('Complete Run: Velocity Tracking Performance', fontweight='bold', fontsize=13)
axes_summary[0].legend(loc='upper right')
axes_summary[0].grid(True, alpha=0.3)

# Plot 2: Corruption visualization
valid_mask = ~np.isnan(measurement)
axes_summary[1].scatter(t[valid_mask], measurement[valid_mask],
                       c='red', s=5, alpha=0.4, label='Corrupted Measurements')
axes_summary[1].plot(t, true_vel, 'g-', linewidth=2, alpha=0.5, label='True Velocity')
axes_summary[1].plot(t, estimate, 'b-', linewidth=1.5, alpha=0.8, label='Filtered Estimate')
axes_summary[1].set_ylabel('Velocity (m/s)')
axes_summary[1].set_title('Sensor Corruption (Noise, Bias, Dropouts) + Kalman Filtering',
                         fontweight='bold', fontsize=13, color='red')
axes_summary[1].legend(loc='upper right')
axes_summary[1].grid(True, alpha=0.3)

# Plot 3: Control signal
axes_summary[2].plot(t, control, 'purple', linewidth=1.5, alpha=0.8, label='PID Control')
axes_summary[2].axhline(y=0, color='k', linestyle=':', alpha=0.3)
axes_summary[2].set_ylabel('Control Signal')
axes_summary[2].set_title('PID Controller Output', fontweight='bold', fontsize=13)
axes_summary[2].legend(loc='upper right')
axes_summary[2].grid(True, alpha=0.3)

# Plot 4: Tracking error analysis
tracking_error = setpoint - estimate
axes_summary[3].plot(t, tracking_error, 'r-', linewidth=1.5, alpha=0.8, label='Tracking Error')
axes_summary[3].axhline(y=0, color='k', linestyle='-', alpha=0.5)
axes_summary[3].fill_between(t, 0, tracking_error, alpha=0.2, color='red')
axes_summary[3].set_xlabel('Time (s)', fontsize=11)
axes_summary[3].set_ylabel('Error (m/s)')
axes_summary[3].set_title('Tracking Error (Setpoint - Estimate)', fontweight='bold', fontsize=13)
axes_summary[3].legend(loc='upper right')
axes_summary[3].grid(True, alpha=0.3)

plt.show()

# Compute statistics
if len(t) > 0:
    mae = np.mean(np.abs(tracking_error))
    rmse = np.sqrt(np.mean(tracking_error**2))
    dropout_count = np.isnan(measurement).sum()
    
    print("\n" + "="*60)
    print("PERFORMANCE STATISTICS")
    print("="*60)
    print(f"\nTracking Performance:")
    print(f"  Mean Absolute Error (MAE): {mae:.4f} m/s")
    print(f"  Root Mean Square Error (RMSE): {rmse:.4f} m/s")
    print(f"\nData Quality:")
    print(f"  Sensor dropouts handled: {dropout_count} / {len(measurement)} ({100*dropout_count/len(measurement):.1f}%)")
    print(f"  Total samples collected: {len(t)}")
    print(f"  Test duration: {t[-1]:.1f} seconds")
    print("\n" + "="*60 + "\n")

## Shutdown

Clean shutdown of all systems.

In [ ]:
print("\n" + "="*60)
print("SHUTTING DOWN")
print("="*60)

# Stop keyboard listener
if KEYBOARD_AVAILABLE:
    try:
        listener.stop()
        print("  ✓ Keyboard listener stopped")
    except:
        pass

# Stop all ROS nodes
print("\nStopping ROS nodes...")
for name, node in nodes:
    try:
        node.stop()
        print(f"  ✓ {name} stopped")
    except Exception as e:
        print(f"  ⚠ {name}: {e}")

# Stop Gazebo
print("\nStopping Gazebo...")
stop_gazebo(gz)

# Destroy nodes
print("\nCleaning up...")
for name, node in nodes:
    try:
        node.destroy()
    except:
        pass

print("\n" + "="*60)
print("SHUTDOWN COMPLETE")
print("="*60)
print("\n✓ All systems stopped successfully")

---

## Navigation

**[Previous: ROS Deployment](ros_deployment.ipynb)**

**← [Back to What is PyKal?](../what_is_pykal/index.rst)**

*This is the final notebook in the PyKal tutorial series.*

## Conclusion: The Complete PyKal Pipeline

You've just completed the **full robotics development cycle**:

### 1. Theory → Software
- Designed control system as composable `DynamicalSystem` components
- Same code works everywhere: simulation, ROS, hardware

### 2. Software → Simulation  
- Tested with clean data (`car_cruise_control.ipynb`)
- Added realistic corruption (`simulating_the_curse.ipynb`)
- Built data preparation pipeline (`pykal.data.prepare`)

### 3. Simulation → ROS
- Wrapped in `ROSNode` for distributed deployment (`ros_deployment.ipynb`)
- **Same corruption/preparation code** - no changes needed
- Added robustness features (diagnostics, heartbeat, staleness)

### 4. ROS → Hardware Simulation (Gazebo)
- Real physics simulation with TurtleBot3
- Interactive keyboard control
- Live performance monitoring
- **Production-ready architecture**

## Key Insights

**The corruption and preparation pipeline transfers directly:**
```python
# In simulation (pure Python):
yk = corrupt.with_gaussian_noise(yk_true, std=0.03)
yk = prepare.with_calibration(yk, offset=0.02)

# In ROS node callback:
yk = corrupt.with_gaussian_noise(yk_true, std=0.03)  # Same code!
yk = prepare.with_calibration(yk, offset=0.02)      # Same code!

# On real hardware:
# Remove corruption, keep preparation - it just works!
```

**ROSNode handles infrastructure, DynamicalSystem handles domain:**
- `ROSNode`: Topic communication, staleness, diagnostics, heartbeat
- `DynamicalSystem`: State evolution, observations, control logic
- `pykal.data`: Corruption (testing), Preparation (robustness)

## What Makes This Special?

1. **Same Code, Every Stage**: Theory → Software → Sim → ROS → Hardware
2. **Test Corruption in Sim**: Find preparation strategies before hardware
3. **Incremental Validation**: Each stage builds confidence
4. **Production Ready**: Diagnostics, monitoring, graceful degradation
5. **Interactive Development**: Keyboard control, live plots, rapid iteration

## Next Steps to Real Hardware

1. **Characterize Your Sensors**
   - Run the robot, log raw sensor data
   - Measure actual noise, bias, dropout rates
   - Update `pykal.data.corrupt` parameters to match

2. **Validate Preparation Pipeline**
   - Test with realistic corruption in Gazebo
   - Verify filtering performance matches expectations
   - Tune Kalman filter `Q` and `R` matrices

3. **Deploy to Hardware**
   - **Remove corruption node** (real sensors already corrupted!)
   - **Keep preparation pipeline** (calibration, dropout handling, etc.)
   - **Keep diagnostics** (monitor message rates, latency, errors)
   - Same `ROSNode` code, just remove corruption callback

4. **Iterate**
   - Hardware will still surprise you
   - But you have the tools to handle it
   - Adjust preparation pipeline based on real data
   - Re-test in Gazebo, re-deploy to hardware

## The Beauty of PyKal

**Write once, test in simulation, deploy to hardware.**

The same code runs everywhere:
- ✅ Jupyter notebooks (rapid prototyping)
- ✅ Python scripts (offline testing)  
- ✅ Gazebo simulation (physics validation)
- ✅ ROS distributed system (architecture testing)
- ✅ Real robots (production deployment)

You're ready for hardware. **Go build something amazing!** 🚀

---

*This concludes the PyKal tutorial series. For more information:*
- *Documentation: https://pykal.readthedocs.io*
- *GitHub: https://github.com/yourusername/pykal*
- *Algorithm Library: Check the docs for implementations of advanced filters (EKF, UKF, particle filters)*